In [1]:
import pandas as pd
import numpy as np
import glob

In [2]:
## Merge students data files
### Raw data "students to lamas" report from sapians

path = r'Data/Students data/raw'
students_files = glob.glob(path + "/*.xlsx")

li = []
for filename in students_files:
    df = pd.read_excel(filename, index_col=None, header=0)
    li.append(df)
students = pd.concat(li, axis=0, ignore_index=True)

c:\Users\yoave\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\yoave\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\yoave\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
c:\Users\yoave\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [3]:
### Clean data

### Rename columns
new_names = ["id","last_name","first_name","birth_date","birth_year","country_born","aliya_year","gender","fam_status","leom","shnot_limud","address","yeshuv","phone","phone2","father_name","mosad_id","mosad_name","mahoz","peula_id","start_date_p","end_date_p","start_date","end_date","shaot_iun","shot_maas","student_status","status_date","maslul","tehhum","megama","kvuza_id","kita","uhlusia","vouchers"]
students.columns = new_names

### Drop columns
keep_cols = ["id","last_name","first_name","gender","leom","yeshuv","mosad_id","mosad_name","peula_id","start_date_p","end_date_p","start_date","end_date","student_status","maslul","megama","kvuza_id"]
students = students[keep_cols]


In [4]:
## Keep relevant maslulim
### 5=shikum, 6=shabas , 13 = hashmal_mesuyag, 15 = OJT, [21,31,67] = hishtalmut, 34 = patuah , 33= yarok , 37= hashlamat haskala
drop_maslulim = [5,6,13,15,21,31,67,34,33,37]
students = students[~students['maslul'].isin(drop_maslulim)]

## Keep relevant status
drop_statusim = ["                 עזב","        בוגר - מסיים","      עבר לקורס חדש"]
students = students[~students['student_status'].isin(drop_statusim)]


In [5]:
## Date cols to pd.date
students["end_date"] = pd.to_datetime(students["end_date"], format='%d/%m/%Y')
students["end_year"] = students["end_date"].dt.year

In [6]:
## Reshape Students Data
df_s = (students.groupby(["id"])['end_date'].unique()
                   .apply(pd.Series, 1))

df_s.columns =  ['end_date_%s' % (x+1) for x in df_s.columns]

In [23]:
df = pd.merge(students,df_s, on='id', how='left')

In [27]:
## Keep only last end_date
date_cols=["end_date","end_date_1","end_date_2","end_date_3","end_date_4","end_date_5","end_date_6"]
df[date_cols] = df[date_cols].apply(pd.to_datetime, format='%d/%m/%Y')

df = df[["id","end_date","end_date_1","end_date_2","end_date_3","end_date_4","end_date_5","end_date_6"]]
df = df.groupby(["id"]).agg({"end_date": "max", "end_date_1": "max", "end_date_2": "max", "end_date_3": "max", "end_date_4": "max", "end_date_5": "max", "end_date_6": "max"}).reset_index()


In [29]:
df.to_csv("Data/Students data/students_processed.csv", index=False)